In [1]:
using Pickle
using JSON
using NetPricing

Precompiling NetPricing
  ✓ NetPricing
  1 dependency successfully precompiled in 5 seconds. 98 already precompiled.


In [2]:
# d_tra = Pickle.load(open("../tmp/data/000001-000000-000075-d30-08-T.pkl", "r"));
# d_tra = JSON.parsefile("../tmp/data/000001-000000-000075-d30-08-T.json");
# d_res = JSON.parsefile("../tmp/data/000001-000000-000075-d30-08-R.json");


In [6]:
# d_tra["000073-100-2-2-2-1-1-1-1-1500-d30-08"]["A"]
# d_tra["TA"]

Dict{String, Any} with 164 entries:
  "32"  => 31
  "29"  => 29
  "1"   => 1
  "54"  => 53
  "78"  => 77
  "81"  => 80
  "101" => 99
  "2"   => 2
  "105" => 103
  "109" => 107
  "74"  => 73
  "41"  => 40
  "132" => 130
  "153" => 151
  "65"  => 64
  "155" => 153
  "51"  => 50
  "53"  => 52
  "106" => 104
  "119" => 117
  "27"  => 27
  "75"  => 74
  "42"  => 41
  "33"  => 32
  "28"  => 28
  ⋮     => ⋮

In [7]:
tvals = []
for element in d_res
    if element["id"]=="000073-100-2-2-2-1-1-1-1-1500-d30-08"
        tvals = element["tvals"]
    end
end


In [34]:
# Sachant la correspondance entre index avant et apres pour les arcs controlles
# Reconstruire N de meme dimension que tvals, mais il est possible que tvals ne soit pas
# de la bonne dimension, 
Ngamma = [
    tvals[
            d_tra["TA"][string(i)]
        ] 
    for i in range(1, length(tvals))
]
println(length(Ngamma))

Ngamma = [Ngamma; zeros(length(d_tra["TA"]) - length(N1))]
println(length(Ngamma))


# trouver le N original


N_min = min.(Ngamma, N)



162
164


LoadError: UndefVarError: `v1` not defined

In [36]:
using NetPricing
abstract type Formulation end
function MAN!(forms::Vector{<:Formulation}, silent=false, threads=nothing, sdtol=1e-10, kwargs...)
    prob = problem(first(forms))
    # Big M
    Ms = [calculate_bigM(form, threads=threads) for form in forms]
    N = max.(collect.(maximum.(Ms, dims=2))...)
    return Ms, N
end

MAN! (generic function with 4 methods)